In [14]:
import pandas as pd
import requests
from bs4 import BeautifulSoup  #HTML parsing library
from flask import Response

In [17]:
#list variables to store scrapped data

Movie_name = []
Genre = []
Year = []
Time = []
Rating = []
Votes = []
Metascore = []
Rank = []
Gross = []
Director = []
Actors = []
Short_desc = []

In [18]:
#HTML pages  from where we will fatch movies data
pages= ["1","101",'201','301','401','501','601','701','801','901']

In [19]:
for page in pages:
    url = "https://www.imdb.com/search/title/?groups=top_1000&sort=user_rating,desc&count=100&start=" + page + "&ref_=adv_nxt"
    respose = requests.get(url) #Making a request to a web page
    soup = BeautifulSoup(respose.content, "html.parser") #pulling data from IMDB web-page
    movie_data = soup.findAll("div", attrs = {'class':'lister-item mode-advanced'}) #Scrapping data that are under specific class
    
    for row in movie_data: #This loop will fatch movie data one by one and append it into list.
        Movie_name.append(row.h3.a.text)
        Genre.append(row.p.find('span', class_ = 'genre').text[1:])
        Year.append(row.h3.find('span', class_ = 'lister-item-year text-muted unbold').text[-5:-1])
        Time.append(row.p.find('span', class_ = 'runtime').text[:-4])
        Rating.append(row.strong.text)

        val = row.findAll("span", attrs = {'name':'nv'})
        Votes.append(val[0].text.replace(",",""))
        Gross.append(val[1].text[1:-1] if len(val)>1 else "")    

        Metascore.append(row.find('span', class_ = 'metascore').text.strip() if row.find('span', class_ = 'metascore') else "" )

        Director.append(row.findAll('a', attrs={'class':''})[13].text)
    

    

In [20]:
#creating DataFrame from all list variables in which we stored movie information.
df = pd.DataFrame({"Movie_name" : Movie_name,
              "Genre" : Genre,
              "Year" : Year,
              "Time" : Time,
              "Rating" : Rating,
              "Votes" : Votes,
              "Gross(Million_$)" : Gross,
              "Metascore" : Metascore,
              "Director" : Director})

In [21]:
#cheking data types 
df.dtypes

Movie_name          object
Genre               object
Year                object
Time                object
Rating              object
Votes               object
Gross(Million_$)    object
Metascore           object
Director            object
dtype: object

In [22]:
#casting data from object to numeric
df["Year"] = pd.to_numeric(df["Year"])
df["Time"] = pd.to_numeric(df["Time"])
df["Rating"] = pd.to_numeric(df["Rating"])
df["Votes"] = pd.to_numeric(df["Votes"])
df["Metascore"] = pd.to_numeric(df["Metascore"])
df["Gross(Million_$)"] = pd.to_numeric(df["Gross(Million_$)"])


In [23]:
df.dtypes

Movie_name           object
Genre                object
Year                  int64
Time                  int64
Rating              float64
Votes                 int64
Gross(Million_$)    float64
Metascore           float64
Director             object
dtype: object

In [24]:
df.isnull().sum()

Movie_name            0
Genre                 0
Year                  0
Time                  0
Rating                0
Votes                 0
Gross(Million_$)    154
Metascore           164
Director              0
dtype: int64

In [26]:
#Exporting all fatched data into CSV file
df.to_csv("Imdb_clean.csv")